# Predict credit default

- Dataset: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients
- Inspiration for model: https://www.kaggle.com/code/mahyar511/payment-default-prediction-neural-network

# 1. Setup

In [1]:
# ! conda install keras -y
# ! conda install tensorflow -y

In [6]:
# ! conda install xlrd -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.14.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/studio-lab-user/.conda/envs/default

  added / updated specs:
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2022.6.15          |     pyhd8ed1ab_1         154 KB  conda-forge
    xlrd-2.0.1                 |     pyhd8ed1ab_3          92 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         246 KB

The following NEW packages will be INSTALLED:

  xlrd               conda-forge/noarch::xlrd-2.0.1-pyhd8ed1ab_3

The following packages will be UPDATED:

  certifi            conda-forge/linux-64::certifi-2022.6.~ --> conda-forge/noarch::certifi-

In [7]:
# imports
import pandas as pd
import numpy as np

from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical


In [8]:
# get the dataset from UCI ML Repository
# ! curl -o default.xls https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

In [9]:
# load the dataset
df = pd.read_excel('data/default.xls', header=1)
df.shape

(30000, 25)

In [10]:
# examine
df.head(2)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1


In [11]:
# distribution of default
df['default payment next month'].value_counts()

0    23364
1     6636
Name: default payment next month, dtype: int64

In [12]:
# Calculating default Ratio
non_default = len(df[df['default payment next month']==0])
default = len(df[df['default payment next month']==1])
ratio = float(default/(non_default+default))
print('Default Ratio: ', ratio)

Default Ratio : 0.2212


## Feature engineering

In [39]:
# split into input (X) and output (y) variables
predictors = df.drop(['ID', 'default payment next month'], axis=1)
print(predictors.shape[1])

23


In [14]:
# Standardize the predictors
X = StandardScaler().fit_transform(predictors)

In [15]:
y = to_categorical(df['default payment next month'])

In [16]:
print(X.shape, y.shape)

(30000, 23) (30000, 2)


## Train test split

In [17]:
# split into 67% for train and 33% for test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
# how many features are there? (this will be the input dimensions)
X_train.shape

(20100, 23)

In [19]:
# how many in each category now?
print('y_train')
unique, counts = np.unique(y_train, return_counts=True)
for x in [0,1]:
    print(unique[x],counts[x])
print('y_test')
unique, counts = np.unique(y_test, return_counts=True)
for x in [0,1]:
    print(unique[x],counts[x])


y_train
0.0 20100
1.0 20100
y_test
0.0 9900
1.0 9900


# 2. Define Keras Model

In [20]:
# parameters
n_cols = predictors.shape[1]
early_stopping_monitor = EarlyStopping(patience=2)
class_weight = {0:ratio, 1:1-ratio}
print(n_cols)

23


In [21]:
# keras has two APIs: Sequential and Functional.
model = Sequential()

2022-08-30 18:17:44.682604: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
# input layer and first hidden layer
# relu = rectified linear unit
model.add(Dense(12, activation='relu', 
                input_shape = (n_cols,))
         )

In [23]:
# second hidden layer
model.add(Dense(12, activation='relu'))

In [24]:
# output layer
model.add(Dense(2, activation='softmax'))

# 3. Compile Keras Model

In [25]:
# compile the keras model
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']
             )

# 4. Fit the model

In [27]:
# fit the keras model on the dataset

model.fit(X_train, 
          y_train, 
          validation_data=(X_test,y_test), 
          epochs=10, 
          batch_size=10,
          class_weight=class_weight,
          callbacks = [early_stopping_monitor]
         )

Epoch 1/10
2010/2010 [==============================] - 2s 1ms/step - loss: 0.1973 - accuracy: 0.7689 - val_loss: 0.5718 - val_accuracy: 0.7579
Epoch 2/10
2010/2010 [==============================] - 2s 1ms/step - loss: 0.1958 - accuracy: 0.7636 - val_loss: 0.5633 - val_accuracy: 0.7589
Epoch 3/10
2010/2010 [==============================] - 2s 1ms/step - loss: 0.1945 - accuracy: 0.7635 - val_loss: 0.5721 - val_accuracy: 0.7588
Epoch 4/10
2010/2010 [==============================] - 2s 1ms/step - loss: 0.1935 - accuracy: 0.7604 - val_loss: 0.5762 - val_accuracy: 0.7449


# 5. Evaluate

In [28]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

310/310 [==============================] - 0s 906us/step - loss: 0.5762 - accuracy: 0.7449
Accuracy: 74.49


In [29]:
# make probability predictions with the model (they come in pairs)
y_probs = model.predict(X_test)
y_probs[:10]

array([[0.5796942 , 0.42030582],
       [0.7665863 , 0.23341373],
       [0.65597427, 0.3440257 ],
       [0.64649314, 0.35350686],
       [0.42346358, 0.5765364 ],
       [0.5311701 , 0.46882984],
       [0.3668022 , 0.6331979 ],
       [0.40037403, 0.599626  ],
       [0.7683754 , 0.23162456],
       [0.67187417, 0.3281258 ]], dtype=float32)

In [30]:
# express these as booleans
y_probs > 0.7

array([[False, False],
       [ True, False],
       [False, False],
       ...,
       [ True, False],
       [False, False],
       [ True, False]])

In [31]:
# make class predictions with the model
y_preds = (y_probs > 0.5).astype(int)
y_preds[:10]

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0]])

In [32]:
# check out the first ten
print(y_preds[:10].tolist())
print(y_test[:10].tolist())

[[1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0]]
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]]


In [33]:

# Evaluate the model
print(metrics.classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.88      0.78      0.83      7742
           1       0.44      0.63      0.52      2158

   micro avg       0.74      0.74      0.74      9900
   macro avg       0.66      0.70      0.67      9900
weighted avg       0.79      0.74      0.76      9900
 samples avg       0.74      0.74      0.74      9900



# 6. Make Predictions on new data


In [38]:
new_data = X_test[[0]]
model.predict(new_data)

array([[0.5796942, 0.4203058]], dtype=float32)

# 7. Save the model
https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [28]:
model.save("credit-model-1.h5")

In [29]:
# load model
from keras.models import load_model
model2 = load_model('credit-model-1.h5')

In [30]:
# make class predictions with the model
predictions = (model.predict(X) > 0.5).astype(int)
predictions[:10]

array([[0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0]])